In [1]:
#import mlflow
#import mlflow.pyfunc
import pandas as pd
#import mlflow.onnx
import onnx
import onnxruntime
import json
import numpy as np


In [12]:
# For testing only data

features = ["0","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24"]

args = {"0":1.0,"1":1.0,"2":1.0,"3":1.0,"4":1.0,"5":1.0,"6":1.0,"7":1.0,"8":1.0,"9":1.0,"10":1.0,"11":1.0,"12":1.0,"13":1.0,"14":1.0,"15":1.0,"16":1.0,"17":1.0,"18":1.0,"19":1.0,"20":1.0,"21":1.0,"22":1.0,"23":1.0,"24":1.0}

model_path = onnx.load("/home/cdsw/cvsmodel/model.onnx").SerializeToString()

def predictFunc(args):
    #Load data
    filtArgs = {key: [args[key]] for key in features}
    data = pd.DataFrame.from_dict(filtArgs)
    
    #Load model
    #predict = mlflow.pyfunc.spark_udf(spark, "/home/cdsw/cvsmodel")
    #model = mlflow.onnx.load_model("/home/cdsw/cvsmodel")
    so = onnxruntime.SessionOptions()
    so.add_session_config_entry('model.onnx', 'ONNX')
    #session =onnxruntime.InferenceSession('/home/cdsw/cvsmodel/model.onnx',None)
    session =onnxruntime.InferenceSession(model_path,None)

    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name
    #print(input_name)
    #print(output_name)
    args_list=[[*args.values()]]
    #print(args_list)
    data=json.dumps({'domain_tokens' : args_list})
    data=np.array(json.loads(data)['domain_tokens']).astype('float32')
    #print(data)
    result = session.run([output_name], {input_name: data})
    prediction=np.array(result).squeeze()
    #print(np.array(result).squeeze())
    #print(type(prediction))
    #print(np.array(result).squeeze())
    
    return float(prediction)
    

predictFunc(args)

b'\x08\x07\x12\x07tf2onnx\x1a\r1.10.1 a37f29:\xf0\xd8\x03\n\xd9\x01\x12NStatefulPartitionedCall/sequential_20/lambda_19/random_uniform/RandomUniform:0\x1aLStatefulPartitionedCall/sequential_20/lambda_19/random_uniform/RandomUniform"\rRandomUniform*\x0c\n\x05dtype\x18\x01\xa0\x01\x02*\x0e\n\x04seed\x15\x00\x00\x00\x00\xa0\x01\x01*\x0c\n\x05shape@\x01\xa0\x01\x07\n\x8e\x01\n\rdomain_tokens\x126StatefulPartitionedCall/sequential_20/embedding/Cast:0\x1a4StatefulPartitionedCall/sequential_20/embedding/Cast"\x04Cast*\t\n\x02to\x18\x06\xa0\x01\x02\n\xfc\x01\n\'Func/StatefulPartitionedCall/input/_1:0\n6StatefulPartitionedCall/sequential_20/embedding/Cast:0\x12BStatefulPartitionedCall/sequential_20/embedding/embedding_lookup:0\x1a@StatefulPartitionedCall/sequential_20/embedding/embedding_lookup"\x06Gather*\x0b\n\x04axis\x18\x00\xa0\x01\x02\n\xd4\x01\nBStatefulPartitionedCall/sequential_20/embedding/embedding_lookup:0\n\x11const_fold_opt__7\x129StatefulPartitionedCall/sequential_20/flatten_6/Res

3.881858901877422e-06

In [26]:
import numpy as np
a=json.dumps({'data' : [1.0,2.0,3.0]})
a = np.array(json.loads(a)['data']).astype('float32')
a

array([1., 2., 3.], dtype=float32)